# Deep Q Learning (DQN)


## Table of Contents
1. [Markov Decision Process](#mdp)
    1. [Discounted Return](#discounted)
    2. [Policies](#Policies)
    3. [Q-Value](#qval)
    4. [Bellman Optimality Equation](#bellman)
2. [Q-Learning](#qlearning)
    1. [The Lizard Game](#lizard)
    2. [Epsilon Greedy Strategy](#egs)
    3. [Q-Value Update](#qupdate)

## Markov Decision Process
<a id='mdp'></a>

MDP is a discrete-time stochastic control process. It provides a mathematical framework for modeling decision making in situations where outcomes are partly random and partly under the control of a decision maker. 
The main elements of MDP are:
* Environment
* Agent. A decision-maker interacting with the environment performing subsequent actions.
* States. Representations of the environment under certain conditions.
* Action. Performed by the agent with respect to the state.
* Reward. Consequence of the action given to the agent.

Given a finite set of states $\mathbf{S}$, a finite set of actions $\mathbf{A}$, and a finite set of rewards $\mathbf{R}$, At each time step $t=0, 1, 2, ...$ the agent receives some representation of the environment’s state $s_t \in \mathbf{S}$. Based on this state, the agent selects an action $a_t \in \mathbf{A}$ resulting in the state-action pair $(s_t, a_t)$.

Time is then incremented to the next time step $t+1$, and the environment transits to a new state $s_{t+1} \in s_{t}$. At this time, the agent receives a numerical reward $r_{t+1} \in \mathbf{R}$ for the action $a_{t}$ performed in the state $s_{t}$.

The reward assignment is represented by a function $f(s_t, a_t) = r_{t+1}$.

### Discounted Return
<a id='discounted'></a>
The importance of the reward relies on the fact that the goal of the agent is to choose the action that makimizes the cumulative rewards. These rewards can be assessed to through the concept of **expected return** defined as:
\begin{equation}
G\_t = r{t+1}+r\_{t+2}+...+r\_{T}
\end{equation}
where $T$ is the final time step. In this way, the expected return is the sum of the future rewards. However, it may happens that the considered environment cannot evolve in a finit number of time steps, making $T=\infty$. Beacuse of this, the consept of the expected return can be translated to the **discounted return** one. 

This means that the agent will try to perform an action by maximizing the cumulative rewards, but focusing more on the immediate reward over future rewards. The discounted return can be defined as:
\begin{equation}
G\_t = r\_{t+1}+\gamma r\_{t+2}+\gamma^2 r\_{t+3}+...=\sum\_{k=0}^\infty \gamma^kr\_{t+1+k}
\end{equation}
Another formulation of the discounted return is 
\begin{equation}
G\_t = r\_{t+1} + \gamma G\_{t+1}
\end{equation}

$0\leq\gamma<1$ is the discount rate and is the weight assigned to the future rewards.

### Policies
<a id='policies'></a>
To know how likely it is for an agent to take any given action from any given state, the policies are used.

A policiy $\pi$ is afuction that maps a given state to probabilites of selecting each possible action from the state:
\begin{equation}
\pi(a|s) = Pr\{a\_t = a| s\_t = s\}
\end{equation}

### Q-Value
<a id='qval'></a>
By recalling that the rewards an agent expects to receive are dependent on what actions the agent takes in given states, it is possible to define the Q-value as a measure of a state-action pair goodness in terms of expected/discounted returns.
In this way, the state-action Q-value for policy $\pi$, referred as $Q_\pi$, tells how good is for the agent to take any given action from a given state while following plicy $\pi$. And so, $Q_\pi(s,a)$ is the expected return from starting from the state $s$, taking the action $a$ under the policy $\pi$:
\begin{equation}
Q\_\pi(s,a) = E\_\pi[G\_t|s\_t = s, a\_t = s] = E\_\pi\left[ \sum\_{k=0}^\infty \gamma^k r\_{t+1+k}|s\_t = s, a\_t = a\right]
\end{equation}

### Bellman Optimality Equation 
<a id='bellman'></a>
The goal of **reinforcement learning** algorithms is to find a policy that will yield a lot of rewards for the agent if the agent indeed follows that policy. Specifically, reinforcement learning algorithms seek to find a policy that will yield more return to the agent than all other policies.
 
For this reason, in terms of return, a policy $\pi'$ is considered better with respect to another $\pi$ if 
\begin{equation}
\pi' \geq \pi \Leftrightarrow Q\_\pi'(s,a) \geq Q\_\pi \;,\; \forall s \in \mathbf{S},\;\forall a \in \mathbf{A}
\end{equation}
In this way, the optimal policy has an optimal Q-value $Q^*$ defined as:
\begin{equation}
Q^*(s,a) = \underset{\pi}{\max} Q\_\pi(s,a)
\end{equation}

If $Q^*$ is optimal, then the Bellman equation is satisfied:
\begin{equation}
Q^*(s\_t,a\_t) = E\left[r\_{t+1} + \gamma \underset{\pi}{\max} Q\_\pi(s\_{t+1},a\_{t+1}) \right]
\end{equation}
This means that the optimal Q-value referred to an $(s,a)$ pair at time $t$ is the expected return $r_{t+1}$ which can be obtined by choosing the action $a$ from the state $s$ plus the maximum expected discounted return that can be achieved from any possible **next** state-action pairs.

Once $Q^*$ is found, then it is possible to determine the optimal policy because, with $Q^*$, for any state $s$, a reinforcement learning algorithm can find the action that maximizes $Q^*(s,a)$. 

## Q-Learning
<a id='qlearning'></a>
Q-learning is a technique that can solve for the optimal policy in an MDP.

The objective of Q-learning is to find a policy that is optimal in the sense that the expected value of the total reward over all successive steps is the maximum achievable. So, in other words, the goal of Q-learning is to find the optimal policy by learning the optimal Q-values for each state-action pair.

The main idea is to iteratively update the Q-values for each state-action pair using the Bellman equation until the Q-function converges to the optimal Q-function. This approach is called **value iteration**. 

To better understand this let's consider the Lizard Game example.

### The Lizard Game
<a id='lizard'></a>
The agent in the environment is the lizard. The lizard wants to eat as many crickets as possible in the least amount of time without stumbling across a bird, which will, itself, eat the lizard.

The lizard can move left, right, up, or down in this environment. These are the 4 actions. The states are determined by the individual tiles and where the lizard is on the board at any given time.

If the lizard lands on a tile that has one cricket, the reward is +1. Landing on an empty tile is -1. A tile with five crickets is +10 points and will end the episode. A tile with a bird is -1 points and will also end the episode.

```
| State  | 1 Cricket | 5 Crickets | Empty | Bird |
|--------|-----------|------------|-------|------|
| Reward | +1        | +10        | -1    | -10  |
```

Now, at the start of the game, the lizard has no idea how good any given action is from any given state. It’s not aware of anything besides the current state of the environment. In other words, it doesn’t know from the start whether navigating left, right, up, or down will result in a positive reward or negative reward.

Therefore, the Q-values for each state-action pair will all be initialized to zero since the lizard knows nothing about the environment at the start. Throughout the game, though, the Q-values will be iteratively updated using value iteration.
To do this, the Q-table is used. It is a table storing the Q-values for each state-action pair. The horizontal axis of the table represents the actions, and the vertical axis represents the states. So, the dimensions of the table are $|\mathbf{S}|$x$|\mathbf{A}|$.

As just mentioned, since the lizard knows nothing about the environment or the expected rewards for any state-action pair, all the Q-values in the table are first initialized to 0. Over time, though, as the lizard plays several episodes of the game, the Q-values produced for the state-action pairs that the lizard experiences will be used to update the Q-values stored in the Q-table.

As the Q-table becomes updated, in later moves and later episodes, the lizard can look in the Q-table and base its next action on the highest Q-value for the current state. 

In each **episode**, the lizard starts out by choosing an action from the starting state based on the current Q-values in the table. The lizard chooses the action based on which action has the highest Q-value in the Q-table for the current state.

Since in the first episode all the Q-values are set zero, there’s no way for the lizard to differentiate between them to discover which one is considered better. To solve this problem, the **Epsilon Greedy Strategy** is used.

### Epsilon Greedy Strategy
<a id='egs'></a>
By defining **Exploration** as the act of exploring the environment to find out information about it (the lizard in the lizard game chooses randomly the action) and **Exploitation** as the act of exploiting the information that is already known about the environment in order to maximize the return (the lizard chooses on the basis of the Q-table), the Epsilon Greedy Strategy (EGD) allows to balance between exploitation and exploration. 

It relies on the exploration rate $\epsilon$ initially set to 1. This exploration rate is the probability that the agent will explore the environment rather than exploit it. By setting $\epsilon = 100\%$ at the beginning, for sure the agent will randomly choose the actions.

As the agent learns more about the environment, at the start of each new episode, will decay by some rate so that the likelihood of exploration becomes less and less probable as the agent learns more and more about the environment. The agent will become “greedy” in terms of exploiting the environment once it has had the opportunity to explore and learn more about it. 

The best practice consists on randomly generating a number $0 \leq x \leq 1$. If $x > \epsilon$ then exploit, otherwise explore.
    
### Q-Value Update
<a id='qupdate'></a>
To update the Q-value for the action taken from the previous state, the Bellman equation is used.
    
It is requested that the Q-value for the given state-action pair is as close as possible to the right hand side of the Bellman equation so that the Q-value will eventually converge to the optimal one. This will happen over time by iteratively comparing the loss between the obtained Q-value $Q(s,a)$ and the optimal one $Q^*(s,a)$ for the given state-action pair and then updating the Q-value over and over again each time this same state-action pair to reduce the loss is encountered.

To do this, the **learning rate** $0 \leq \alpha \leq 1$ is used. It can be thought of as how quickly the agent abandons the previous Q-value in the Q-table for a given state-action pair for the new Q-value.

So, for example, by assuming to have a Q-value in the Q-table for some arbitrary state-action pair that the agent has experienced in a previous time step. Well, if the agent experiences that same state-action pair at a later time step once it's learned more about the environment, the Q-value will need to be updated to reflect the change in expectations the agent now has for the future returns.

Instead of just overwriting the old Q-value, the learning rate is used as a tool to determine how much information must be kept about the previously computed Q-value for the given state-action pair versus the new Q-value calculated for the same state-action pair at a later time step. In this way, the new Q-value can be determined as:
    \begin{equation}
        Q'(s,a) = (1-\alpha)Q(s,a) + \alpha\left(r\_{t+1} + \gamma \underset{a\_{t+1}}{\max}Q(s\_{t+1}, a\_{t+1}) \right)
    \end{equation}
where $Q(s,a)$ is the old value and $r_{t+1} + \gamma \underset{a_{t+1}}{\max}Q(s_{t+1}, a_{t+1}$ is the lerned one. Thus the learned value is the reward the agent receives from performing the action $a_t$ from the starting state $s_t$ plus the discounted estimate of the optimal future Q-value for the next state-action pair $(s_{t+1}, a_{t+1})$.

## Deep Q-Learning
<a id='dql'></a>

Deep Q-Learning consists on generalizing the Q-Learning problems by applying neural networks. It is well suited for machine learning problems in which the state space is too large to be stored in a Q-table and when a short dataset is available for training.  

By exploiting the bellman equation and the epsilon greedy strategy, indeed, it is possible to create a dataset composed by the randomly discovered state. This dataset is then used as input to train a neural network whose output is the expected discounted return associated to each possible (state, action) pairs.  

Before describing the training process, it is necessary to define the Replay Memory.

### Replay Memory
<a id='replay'></a>

Replay memory relies on the concept of experience which can be considered as what the agent learned from the previous state of the environment. More specifically the experience acquired at time $t$ can be defined as:

\begin{equation}
e\_t = (s\_t, a\_t, r\_{t+1}, s\_{t+1})
\end{equation}
this means that it is a collection of the current environment state, the performed action, the resulting reward and the next state in which the agent is after having performed the action.  

The current experience of each step is stored in a memory with finite capacity $N$. This memory randomly sampled to remove the correlation between the subsequent steps is used as the training dataset of a neural network.

### Deep Q Networks
Deep Q Networks (DQN) are used in DQL to estimate the action resulting in the optimal Q-value given an input state. This means that the loss function of the network can be defined as the difference between the predicted Q-value for each (state, action) pair and the optimal one obtained through the Bellman equation:
\begin{equation}
loss = Q^*(s,a) - Q(s,a)
\end{equation}
Now by considering the definition of $Q^*(s,a)$ which depends on the current (state,action) pair and on the Q-value of the future (state, action) pairs, it is possible to draw a scheme of the Deep Q Networks iterations in this way:

1. Initialize an empty replay memory capacity.
2. Initialize the network with random weights.
3. For each episode:
    1. Initialize the starting state.
    2. For each time step:
        1. Get the current state.
        2. Select an action via exploration or exploitation (EGS).
        3. Perform the selected action.
        4. Get the reward.
        5. Get the next state
        6. Update the memory with the acquired experience.
        7. Train the network through the 'Replay'

Moreover to train the network through the 'Replay' the following process can be implemented:
1. Predict the Q-values of the next state taken from the experience referred to each possible actions $Q(s_{t+1},a)$ $\forall a \in \mathbf{A}$.
2. Get the action $a^*$ leading the greatest $\underset{a}{\max}Q(s_{t+1}, a$ Q-value
3. Compute the optimal Q-value $Q^*(s_{t+1},a)$ through the Bellman equation
4. Predict the Q-values of the current state taken from the experience referred to each possible actions $Q(s_t,a)$ $\forall a \in \mathbf{A}$.
5. Replace the obtained Q-value of the action taken at 2. with the optimal one $Q(s_t,a^*) = Q^*(s_{t+1},a)$
6. Use the set of Q-values predicted at 4. with the modification of 5. as target variable for the neural network and train it.

At this point, after a sufficient number of episodes, 